##### Short Version

Stardew Valley is an ingenious little "farming simulator" among many other things.. In order to maximize my profits, I want to calculate the compound return of crops where you buy seeds, harvest crops over a certain duration, sell those crops and reinvest the earnings into more seeds.

There are two types of crops (broadly) so I've written two subclasses to take in these different types of crops and calculate a compound calculation.

##### Long Version
I've decided to write a class to help me in a goofy little video game I like to play, Stardew Valley. Stardew Valley is a simple little "farming" simulation game in the vein of Harvest Moon. That's putting it simplistically... but this isn't Stardew Apologetics. Wikipedia can explain it better than me: https://en.wikipedia.org/wiki/Stardew_Valley

One of the primary features of this game is (obviously) farming. You can buy seeds for a variety of crops and plant them with various different growth rates and then sell them at different prices. Being the sort of perfectionist I am at these kinds of silly, resource building games I immediatley wanted to maximize my revenue.

Looking online, I immediately understood from the sites I was visiting that these authors really didn't understand the idea of compounding. When trying to find recommendations on crops during different seasons to focus on to make the most money, all of the articles I read evaluated crops in terms of "coins per day". Essentially, if something like (price-cost)/growth. This sounds right for most people, but I knew immediatley that a better model would be a compounding return model. Essentially, it relates to that old riddle you might have heard when you were a kid: "Would you rather have a million dollars a day, or have a penny double every day for a month". And if anyone does the map, despite that a penny doubling doesn't seem like it would get very far any time soon, you'll quickly realize how quickly it adds up, easily dwarfing a million dollars.

So naturally, I knew that it wasn't simple enough to evaluate crops from a "earnings per day" method when you can reinvest those earnings for duration of 28 days. So I wanted to calculate what that would be for every crop to make an informed decision on which crop to invest in. But, since this is a little made up world, I had to write very specific rules other than the standard compounding formula (which would be easy to write a function for).

There are some fundamentals of the game that make calulating the growth rate of your reinvetestment more complex. For isntance, if the shop is closed, you won't be able to purchase the seeds with the proceeds from your last harvest. So, we'll need a way to calculate what the return will be if your purchase of seeds is delayed by the number of days the shop is closed. Also, there are only real numbers in this game. So we'll need to track remainders left when we try to reinvest our proceeds into seeds that are not without a common factor.

Finally, the last challenge we have in modeling our Crop investing is that there are two different types of crops that effect this class. Firstly, some crops have a set growth duration. Meaning than if you plant them on x day, they will yield a crop x + n days and then the crop terminates at harvest. These types of crops are what I call "nonrenewable". You plant them, you harvest them, and then you must replant them with the proceeds from your harvest. However, some crops are what I call "renewable"; they continue to produce on a defined cycle without having to be replanted. They always have long intial durations for the first harvest, and then have shorter durations for every subsequent harvest.

In [11]:
class Crop:
    
    season_duration = 28
    
    def __init__(self, crop, cost, sell_price, growth_duration, intial_grow=None):
        self.crop = crop
        self.cost = cost
        self.sell_price = sell_price
        self.growth_duration = growth_duration
        self.intial_grow = intial_grow
    
    def get_crop(self):
        return self.crop
    
    def get_cost(self):
        return self.cost
    
    def get_sell_price(self):
        return self.sell_price
    
    def get_growth_duration(self):
        return self.growth_duration
    
    def get_intial_grow(self):
        return self.intial_grow
            
    def shop_closed(self, start=0):
        end = Crop.season_duration
        
        return [x + 1 for x in range(start-1,end) if (x + 1) % 7 == 3]
    

class NonRenewCrop(Crop):
        
    def __init__(self, crop, cost, sell_price, growth_duration):
        super().__init__(crop, cost, sell_price, growth_duration)

        self.leftover_days = Crop.season_duration % self.growth_duration
        self.harvest_count = int((Crop.season_duration - self.leftover_days) / self.growth_duration + 1)
        
    def pay_days(self, start=1):
        end = Crop.season_duration
        
        isharvest = lambda x: x % (self.growth_duration + 1) == 0
        isclosed = lambda x: x % 7 == 3
        
        return [x+start-1 if isharvest(x) and not isclosed(x+1) else x+start for x in range(start,end+1) if isharvest(x)]        

    def compound(self, account=500):
        for x in self.pay_days():
            remainder = account % self.cost
            revenue = ((account - (remainder))/self.cost)*self.sell_price
            account = revenue + remainder
            
        return account
    
    def harvest(self):
        print(self.period)
        print(self.harvest_count)

class RenewCrop(Crop):
        
    def __init__(self, crop, cost, sell_price, growth_duration, intial_grow):
        super().__init__(crop, cost, sell_price, growth_duration, intial_grow)

        self.leftover_days = Crop.season_duration % self.growth_duration
        self.harvest_count = int((Crop.season_duration - self.leftover_days) / self.growth_duration + 1)
        
    def compound(self, account=500):
        for x in 
        
        pass


In [12]:
renewable_crops = [['Parsnip', 20, 35, 4], 
                   ['Potato', 50, 100, 6], 
                   ['Cauliflower', 80, 175, 12], 
                   ['Kale', 70, 110, 6],
                   ['Garlic', 40, 60, 4]]

In [13]:
for crop in renewable_crops:
    crop = NonRenewCrop(crop[0],crop[1],crop[2],crop[3])
    print(crop.crop, crop.compound())
    

Parsnip 8135.0
Potato 8000.0
Cauliflower 2305.0
Kale 2980.0
Garlic 3680.0


Despite the Internet's love of the Potato, from a compound perspective, Parsnips are better for an overall return since their growth_duration (therefore the reinvestment cycle) is shorter.

In [14]:
nonrenewable_crops = [['Green Bean', 60, 40, 3, 10],
                      ['Blueberries', 80, 240, 4, 13],
                      ['Hops', 60, 25, 11, 1],
                      ['Hot Pepper', 40, 40, 3, 5],
                      ['Tomato', 50, 60, 4, 11],
                      ['Corn', 150, 50, 4, 14],
                     ]

for crop in nonrenewable_crops:
    crop = RenewCrop(crop[0],crop[1],crop[2],crop[3],crop[4])
    print(crop.crop, crop.compound())
                     

Green Bean None
Blueberries None
Hops None
Hot Pepper None
Tomato None
Corn None


In [18]:
crop = NonRenewCrop(renewable_crops[0][0], renewable_crops[0][1], renewable_crops[0][2], renewable_crops[0][3])

crop.shop_closed()

[3, 10, 17, 24]